In [2]:
import pandas as pd
import pandas as pd
from datetime import datetime
import seaborn as sns
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import numpy as np

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
import matplotlib.pyplot as plt



In [3]:
df=pd.read_csv("/Users/kmbl227932/Downloads/cv_quarterly_scrub-28062024074843-000.csv")
df=pd.concat([df,pd.read_csv("/Users/kmbl227932/Downloads/cv_quarterly_scrub-28062024074843-001.csv")])
df=pd.concat([df,pd.read_csv("/Users/kmbl227932/Downloads/cv_quarterly_scrub-28062024074843-002.csv")])




In [ ]:
  
# CV Filters
# Step 1- Excluding non banking Financial institutions from the savings account list 


# Step 2- Apply following conditions to filter out bases

#     Type -1 Non fd takers who keep more than 5k with us , but have more than 1 - 1.5 or more times amb in external funds as well

#     Type- 2 Non fd takers who keep 0 amb with us, but have more than 10k in external funds

#     Type -3 Non fd takers who keep 1 to 5k amb with us but have more than 10k in external funds

#     Type -4 Non fd takers who have external AMB >2.33 times Internal AMB and external AMB >5K 
    
# Step 3- Deduping from above 4 categories 
 
 
# New logic for Jun onwards- 
# Instead of AMB we took Credits - Debits of 3 months as the cutoff and included pitch to FD and RD as well.

# Non FD takers with Cr-Dr >5k - Pitch for FD

# Non FD takers with Cr-Dr 1-5k - Pitch for RD


In [4]:
df.columns

Index(['uniqueid', 'crn', 'data.income.data.truesalary',
       'data.customer360.data.metadata.asofdate',
       'data.income.data.predictedincomebucket.confidence',
       'data.customer360.data.bankaccount.primaryaccount',
       'data.customer360.data.bankaccount.aggregateaveragemonthlybalance',
       'data.customer360.data.bankaccount.aggregatedebit.maxdebitamount.month1',
       'data.customer360.data.bankaccount.aggregatedebit.maxdebitamount.month2',
       'data.customer360.data.bankaccount.aggregatedebit.maxdebitamount.month3',
       'data.customer360.data.bankaccount.aggregatecredit.maxcreditamount.month1',
       'data.customer360.data.bankaccount.aggregatecredit.maxcreditamount.month2',
       'data.customer360.data.bankaccount.aggregatecredit.maxcreditamount.month3',
       'data.customer360.data.bankaccount.aggregatecredit.totalcreditamount.month1',
       'data.customer360.data.bankaccount.aggregatecredit.totalcreditamount.month2',
       'data.customer360.data.bankacc

In [5]:
df['cr-db_m1']=df['data.customer360.data.bankaccount.aggregatecredit.totalcreditamount.month1']-df['data.customer360.data.bankaccount.aggregatedebit.totaldebitamount.month1']

df['cr-db_m2']=df['data.customer360.data.bankaccount.aggregatecredit.totalcreditamount.month2']-df['data.customer360.data.bankaccount.aggregatedebit.totaldebitamount.month2']
df['cr-db_m3']=df['data.customer360.data.bankaccount.aggregatecredit.totalcreditamount.month3']-df['data.customer360.data.bankaccount.aggregatedebit.totaldebitamount.month3']




In [6]:
df['data.customer360.data.bankaccount.aggregateaveragemonthlybalance'].describe(percentiles=[.1,.2,.3,.4,.5,.6,.7,.8,.9,.95])

count    7752444.000
mean       29309.045
std       140245.829
min            0.000
10%           89.380
20%          346.190
30%          813.120
40%         1590.532
50%         2939.290
60%         5357.808
70%        10053.911
80%        20098.528
90%        51802.565
95%       113388.082
max     17216507.530
Name: data.customer360.data.bankaccount.aggregateaveragemonthlybalance, dtype: float64

In [ ]:
# 1-5k cr-dr rd
# >5k cr-dr fd


# 50, 25, 5 amb priority  

# remove these bases 
# gold - loan - 1.7-1.8 (20k)
# fd 1
# am 2
# rd 3

In [7]:
fd_base=df[(df['cr-db_m1']>5000)]

In [22]:
fd_base['priority']=fd_base['data.customer360.data.bankaccount.aggregateaveragemonthlybalance'].rank(method='dense',ascending=False,pct=True).astype(int)

/var/folders/ht/mft9wz317xd4cttlzy2ntw403_jv97/T/ipykernel_17461/2273568930.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fd_base['priority']=fd_base['data.customer360.data.bankaccount.aggregateaveragemonthlybalance'].rank(method='dense',ascending=False,pct=True).astype(int)


In [23]:
fd_base[['crn','cr-db_m1','data.customer360.data.bankaccount.aggregateaveragemonthlybalance']].to_csv("fd_cv_propensity_jun.csv")

/var/folders/ht/mft9wz317xd4cttlzy2ntw403_jv97/T/ipykernel_17461/2273568930.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fd_base['priority']=fd_base['data.customer360.data.bankaccount.aggregateaveragemonthlybalance'].rank(method='dense',ascending=False,pct=True).astype(int)


In [10]:
df[(df['cr-db_m1']>5000)]['data.customer360.data.bankaccount.aggregateaveragemonthlybalance'].describe()

count    1207369.000
mean       63229.457
std       209676.737
min            0.000
25%         4826.430
50%        14021.340
75%        43952.170
max     12381587.880
Name: data.customer360.data.bankaccount.aggregateaveragemonthlybalance, dtype: float64

In [11]:
df[(df['cr-db_m1']>5000)].shape

(1207369, 29)

In [12]:
df[(df['cr-db_m2']>5000)].shape

(891094, 29)

In [13]:
df[(df['cr-db_m3']>5000)].shape

(773642, 29)

In [14]:
df[(df['cr-db_m1']>5000)&(df['cr-db_m2']>5000)].shape

(265577, 29)

In [15]:
fd_base['priority'].max()



1

In [16]:
df[(df['cr-db_m1']>5000)]['data.customer360.data.bankaccount.aggregateaveragemonthlybalance'].describe()




count    1207369.000
mean       63229.457
std       209676.737
min            0.000
25%         4826.430
50%        14021.340
75%        43952.170
max     12381587.880
Name: data.customer360.data.bankaccount.aggregateaveragemonthlybalance, dtype: float64

In [17]:
rd_base=df[(df['cr-db_m1']<5000)&(df['cr-db_m1']>1000)]

In [18]:
rd_base['data.customer360.data.bankaccount.aggregateaveragemonthlybalance'].describe()

count     704961.000
mean       16248.124
std        82679.115
min            0.000
25%          987.690
50%         2647.360
75%         8644.630
max     10557894.860
Name: data.customer360.data.bankaccount.aggregateaveragemonthlybalance, dtype: float64

In [19]:
rd_base['priority']=rd_base['data.customer360.data.bankaccount.aggregateaveragemonthlybalance'].rank(method='dense',ascending=False,pct=True).astype(int)


/var/folders/ht/mft9wz317xd4cttlzy2ntw403_jv97/T/ipykernel_17461/3368637554.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rd_base['priority']=rd_base['data.customer360.data.bankaccount.aggregateaveragemonthlybalance'].rank(method='dense',ascending=False,pct=True).astype(int)


In [20]:
rd_base['priority'].describe()

count   704961.000
mean         0.001
std          0.036
min          0.000
25%          0.000
50%          0.000
75%          0.000
max          1.000
Name: priority, dtype: float64

In [21]:
rd_base[['crn','cr-db_m1','data.customer360.data.bankaccount.aggregateaveragemonthlybalance']].to_csv("rd_cv_propensity_jun.csv")